In [1]:
import pandas as pd
import numpy as np
import scipy 

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os
import gc
import pickle

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm,trange

plt.style.use('ggplot')

mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.family"] = "Malgun Gothic"

os.chdir("../../")
root_path = os.getcwd()

raw_file_path = os.path.join(root_path, "Bigcon2020_raw_file")
csv_file_path = os.path.join(root_path, "BigCon_2020/csv_files")
pickle_file_path = os.path.join(root_path, "BigCon_2020/pickle_files")
image_path = 'C:/Users/rears/OneDrive/바탕 화면/dacon_covid19/'

c:\python37\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from itertools import permutations

## Prepare Files

* 필요파일
    * 인접 행정동, 행정동간 거리 파일
    * 행정동별 생활인구 파일 
    * 행정동별 교통편의성 파일

In [3]:
distance_file=[pd.read_csv(os.path.join(csv_file_path,file), index_col=[0]) for file in os.listdir(csv_file_path) if file.startswith('distance')]

In [4]:
real_dist = distance_file[0]
adjacent_dist = distance_file[1]

In [5]:
living_population=pd.read_csv(os.path.join(csv_file_path,'CTGG_HDNG_FLOW.csv'))

In [287]:
floating_population=pd.read_csv(os.path.join(csv_file_path,'CTGG_HDNG_POP.csv'))

In [244]:
convenience_index = pd.read_csv(os.path.join(csv_file_path, 'conv_index_df.csv'), index_col=[0])

In [37]:
# bus_route_num_dtd = pd.read_csv(os.path.join(csv_file_path,"bus_route_num_dtd.csv"), index_col=[0])

# living_population.sample(10)

# convenience_index = pd.read_csv(os.path.join(csv_file_path,'conv_index.csv'))

# convenience_index.loc[:,'conv']=np.sqrt(convenience_index.conv)
# living_population.loc[:,'value'] = np.sqrt(living_population.value)

## MCLP MODELING

___

$\underset{h \subset H}{\operatorname{argmax}} (\sum_{h^* \in h} L(h^*) + C(h^*) \cdot adjL(h^*))$ 

$L(h^*) : h^*$행정동의 생활인구 제곱근  
$C(h^*) : h^*$행정동의 교통편의성 지수 제곱근   
$adjL(h^*): h^*$행정동의 인접행정동들의 생활인구합 제곱근   

___
**Modified Version**

$C_{n,n} \circ W_{n,n} \circ L_{n,1}\ , n= number\ of\ dongs$  
  
___
$C_{n,n} = \{c_{i,j}\}\ s.t\ \ c_{i,j}= \begin{cases}
conv\_in(i,j) & \text{if $i=j$} \\
conv\_out(i,j) & \text{if $i \ne j$}
\end{cases}\ , 1 \le i,j \le n$

$L_{n,1}^{(1)} = \{l_{i,1} :=\ living\ population\ of\ dong_i\}, 1 \le i \le n $  
$L_{n,1}^{(2)} = \{l_{i,1} :=\ living\ population\ of\ dong_i\}, 1 \le i \le n $


$ Double\ Power\ Distance\ Weights $  
$D_{n,n} = \{d_{i,j}\} \ s.t\ \ d_{i,j}= \begin{cases}
\left (1-\left (Dist(h_i,h_j) \over d \right)^2 \right )^2 & \text{if $0 \le Dist(h_i,h_j) \le d $} \\
0 & \text{if $Dist(h_i,h_j) > d$}\end{cases}$

$W_{n,n}^{(k)} = \{w_{i,j}^{(k)}\}\ s.t\ \  w_{i,j}^{(k)} = \begin{cases}
w_{i,j}^{(k-1)} \cdot \left (1-D_{h^{(k-1)},j} \right) & \text{if $i=j$}\\
0 & \text{if $i \ne j$}\end{cases}\ \ where \ h^{(k-1)}:selected\ dong\ in \ (k-1)th\ MCLP\ process$  
$\ (If\ k=1, W_{n,n}^{(k)} = I_{n,n})$  

In [7]:
# 거리별 가중치 계산
def double_power_distance_weight(df, distance):
    shape = df.shape
    names = df.columns.tolist()
    flatten_values = np.concatenate(df.values)
    weights = np.array([(1-(dist/distance)**2) if dist < distance else 0 for dist in flatten_values])
    weights_df = pd.DataFrame(weights.reshape(shape), columns = names)
    weights_df.index = names
    return weights_df

___

In [48]:
def double_power_distance_weight(df=real_dist, distance=3000):
    shape = df.shape
    names = df.columns.tolist()
    flatten_values = np.concatenate(df.values)
    weights = np.array([(1-(dist/distance)**2) if dist < distance else 0 for dist in flatten_values])
    weights_df = pd.DataFrame(weights.reshape(shape), columns = names)
    weights_df.index = names
    return weights_df

In [60]:
HDONG_ORDER = convenience_index.index.tolist()

In [332]:
"""
def MCLP(dist_file, population_file, convenience_file, N=4, covid_class='A', Ages='3059', distance=3000,w1=.6,w2=.4):
    
    HDONG_ORDER = convenience_file.index.tolist()
    
    def double_power_distance_weight(df=dist_file, distance=distance):
        shape = df.shape
        names = df.columns.tolist()
        flatten_values = np.concatenate(df.values)
        weights = np.array([(1-(dist/distance)**2) if dist < distance else 0 for dist in flatten_values])
        weights_df = pd.DataFrame(weights.reshape(shape), columns = names)
        weights_df.index = names
        return weights_df

    def update_weight_matrix(hdong=None, weight_matrix=None, HDONG_ORDER=HDONG_ORDER):
        shape=(34,34)
        if not isinstance(weight_matrix, np.ndarray):
            weight_matrix = np.diag(np.ones((34,)))
            return weight_matrix
        else:
            if hdong not in HDONG_ORDER:
                raise ValueError('hdong must be in HDONG_ORDER')
            
            updated_weight_matrix = weight_matrix*(1-double_power_distance_weight().loc[hdong].values)
            # print(np.diagonal(updated_weight_matrix))
            if updated_weight_matrix.shape==shape:
                return updated_weight_matrix
            else:
                raise ValueError(f'updated_weight_matrix\'s shape : {updated_weight_matrix.shape} is not {shape}')
    
    ### C
    # 교통편의성 matrix 생성
    C = np.asmatrix(convenience_file)
    
    ### W
    # Weighted Maxtrix 초기화(생성) - Identity Matrix
    W = update_weight_matrix()
    
    ### L
    # 생활인구
    # matrix연산 전 행정동의 순서가 제대로 되어있는지 확인 후 생활인구 Matrix 생성
    
    living_population_weekday = population_file.loc[(population_file.Covid_class==covid_class) &
                                           (population_file.variable==Ages)&
                                           (population_file.dayofweek==0),['HDNG_NM','value']]

    living_population_weekend = population_file.loc[(population_file.Covid_class==covid_class) &
                                           (population_file.variable==Ages)&
                                           (population_file.dayofweek==1),['HDNG_NM','value']]

    if living_population_weekday.HDNG_NM.tolist() == HDONG_ORDER:       
        L_1 = np.asmatrix(living_population_weekday.set_index('HDNG_NM'))
        L_1_shape = L_1.shape
    else:
        raise ValueError('[평일]행정동의 순서가 맞지 않습니다')

    if living_population_weekend.HDNG_NM.tolist() == HDONG_ORDER:       
        L_2 = np.asmatrix(living_population_weekend.set_index('HDNG_NM'))
        L_2_shape = L_2.shape
    else:
        raise ValueError('[주말]행정동의 순서가 맞지 않습니다')

    result = []

    tmp_df = pd.DataFrame(HDONG_ORDER, columns=['HDONG_NM'])
    for i in range(N):        
        if i == 0:
            hdong = HDONG_ORDER[np.argmax(w1*(C*W*L_1)+w2*(C*W*L_2))]
            result.append(hdong)
        else:
            W = update_weight_matrix(hdong=hdong, weight_matrix=W)
            hdong = HDONG_ORDER[np.argmax(w1*(C*W*L_1)+w2*(C*W*L_2))]
            result.append(hdong)
        #print(pd.DataFrame(np.diagonal(W)))
        # tmp_df[f'W_{i}'] = np.diagonal(W)
        tmp_df[f'result_{i}'] = w1*(C*W*L_1) + w2*(C*W*L_2)
    # print(tmp_df)
    
        
    return result, tmp_df
"""

"\ndef MCLP(dist_file, population_file, convenience_file, N=4, covid_class='A', Ages='3059', distance=3000,w1=.6,w2=.4):\n    \n    HDONG_ORDER = convenience_file.index.tolist()\n    \n    def double_power_distance_weight(df=dist_file, distance=distance):\n        shape = df.shape\n        names = df.columns.tolist()\n        flatten_values = np.concatenate(df.values)\n        weights = np.array([(1-(dist/distance)**2) if dist < distance else 0 for dist in flatten_values])\n        weights_df = pd.DataFrame(weights.reshape(shape), columns = names)\n        weights_df.index = names\n        return weights_df\n\n    def update_weight_matrix(hdong=None, weight_matrix=None, HDONG_ORDER=HDONG_ORDER):\n        shape=(34,34)\n        if not isinstance(weight_matrix, np.ndarray):\n            weight_matrix = np.diag(np.ones((34,)))\n            return weight_matrix\n        else:\n            if hdong not in HDONG_ORDER:\n                raise ValueError('hdong must be in HDONG_ORDER')\n      

In [324]:
def MCLP(dist_file, population_file, convenience_file, N=4, Ages='3059', distance=3000, w1=.6,w2=.4, covid_weight=[.2,.2,.2,.2,.2]):
    
    HDONG_ORDER = convenience_file.index.tolist()
    
    def double_power_distance_weight(df=dist_file, distance=distance):
        shape = df.shape
        names = df.columns.tolist()
        flatten_values = np.concatenate(df.values)
        weights = np.array([(1-(dist/distance)**2) if dist < distance else 0 for dist in flatten_values])
        weights_df = pd.DataFrame(weights.reshape(shape), columns = names)
        weights_df.index = names
        return weights_df

    def update_weight_matrix(hdong=None, weight_matrix=None, HDONG_ORDER=HDONG_ORDER):
        shape=(34,34)
        if not isinstance(weight_matrix, np.ndarray):
            weight_matrix = np.diag(np.ones((34,)))
            return weight_matrix
        else:
            if hdong not in HDONG_ORDER:
                raise ValueError('hdong must be in HDONG_ORDER')
            
            updated_weight_matrix = weight_matrix*(1-double_power_distance_weight().loc[hdong].values)
            # print(np.diagonal(updated_weight_matrix))
            if updated_weight_matrix.shape==shape:
                return updated_weight_matrix
            else:
                raise ValueError(f'updated_weight_matrix\'s shape : {updated_weight_matrix.shape} is not {shape}')
    
    ### C
    # 교통편의성 matrix 생성
    C = np.asmatrix(convenience_file)
    
    ### W
    # Weighted Maxtrix 초기화(생성) - Identity Matrix
    W = update_weight_matrix()
    
    
    
    tmp_df = pd.DataFrame(HDONG_ORDER, columns=['HDONG_NM'])
    result = [] 
    class_num = population_file.Covid_class.nunique()
        
    for i in range(N):
        result_mat = np.zeros((34,1))
        for covid_idx, covid_class in enumerate(population_file.Covid_class.unique()):
            ### L
            # 생활인구 - 코로나 정도 및 평일/주말로 구분
            # matrix연산 전 행정동의 순서가 제대로 되어있는지 확인 후 생활인구 Matrix 생성
            living_population_weekday = population_file.loc[(population_file.Covid_class==covid_class) &
                                                   (population_file.variable==Ages)&
                                                   (population_file.dayofweek==0),['HDNG_NM','value']]

            living_population_weekend = population_file.loc[(population_file.Covid_class==covid_class) &
                                                   (population_file.variable==Ages)&
                                                   (population_file.dayofweek==1),['HDNG_NM','value']]

            if living_population_weekday.HDNG_NM.tolist() == HDONG_ORDER:       
                L_1 = np.asmatrix(living_population_weekday.set_index('HDNG_NM'))
                L_1_shape = L_1.shape
            else:
                raise ValueError('[평일]행정동의 순서가 맞지 않습니다')

            if living_population_weekend.HDNG_NM.tolist() == HDONG_ORDER:       
                L_2 = np.asmatrix(living_population_weekend.set_index('HDNG_NM'))
                L_2_shape = L_2.shape
            else:
                raise ValueError('[주말]행정동의 순서가 맞지 않습니다')

            if i == 0:    
                result_mat += covid_weight[covid_idx]*(w1*(C*W*L_1)+w2*(C*W*L_2))
                if covid_idx == (class_num-1):
                    hdong = HDONG_ORDER[np.argmax(result_mat)]
                    result.append(hdong)
            else:
                if covid_idx == 0:
                    W = update_weight_matrix(hdong=hdong, weight_matrix=W)
                    
                result_mat += covid_weight[covid_idx]*(w1*(C*W*L_1)+w2*(C*W*L_2))
                if covid_idx == (class_num-1):
                    hdong = HDONG_ORDER[np.argmax(result_mat)]
                    result.append(hdong)

            #print(pd.DataFrame(np.diagonal(W)))
            # tmp_df[f'W_{i}'] = np.diagonal(W)
            if covid_idx == (class_num-1):
                tmp_df[f'result_{i}'] = result_mat
        # print(tmp_df)

        
    return result, tmp_df

In [303]:
result, process = MCLP(real_dist, living_population, convenience_index, N=4)

In [328]:
result_dict = {}
for n in [4,5,6,7,8,10]:
    tmp_dict={}

    result, process = MCLP(real_dist, living_population, convenience_index, N=n)
    
    tmp_dict['result'] = result
    tmp_dict['process'] = process
    
    result_dict[n] = tmp_dict

In [333]:
# with open(os.path.join(pickle_file_path, 'result_dict.pickle'), 'wb') as f:
#     pickle.dump(result_dict,f,protocol=pickle.HIGHEST_PROTOCOL)

In [334]:
r,df = MCLP(real_dist, living_population, convenience_index, N=10)

In [340]:
df.iloc[:,1:].sum().diff().dropna()

result_1   -210095.969449
result_2   -109107.346523
result_3    -78701.957432
result_4    -36041.501870
result_5    -13061.877591
result_6    -13093.319173
result_7    -10527.963324
result_8     -2458.405423
result_9     -5138.645425
dtype: float64

In [274]:
result = {}

for var in living_population.variable.unique():
    result[var] = MCLP(real_dist, living_population, convenience_index, N=7)

In [129]:
HDONGS=adjacent_dist.columns.tolist()
adjacent_dong_dict = {}
    
for i,dong in tqdm(enumerate(HDONGS), desc='Finding Adjacent Dongs'):
    adjacent_dong_dict[dong] = list(adjacent_dist.loc[adjacent_dist[dong]==0,HDONGS[i]].drop(dong).index)

Finding Adjacent Dongs: 34it [00:00, 1407.47it/s]


In [37]:
pop_dict=dict(living_population.loc[(living_population.dayofweek==0) & (living_population.Covid_class=='A') & (living_population.variable=='3059')].set_index('HDNG_NM')['value'])
conv_dict=dict(convenience_index.set_index('HDONG_NM')['conv'])

In [11]:
import copy

In [38]:
def MCLP(dist_file, population_file, convenience_file, distance='adjacent', N=5, covid_class='A', Ages='3059'):
    HDONGS=dist_file.columns.tolist()
    #all_cases = list(combinations(HDONGS,N))
    
    # 평일 생활인구, 주말 생활인구
    weekday_living_pop_dict = dict(population_file.loc[(population_file.dayofweek==0) & (population_file.Covid_class==covid_class) & (population_file.variable==Ages)].set_index('HDNG_NM')['value'])
    weekend_living_pop_dict = dict(population_file.loc[(population_file.dayofweek==1) & (population_file.Covid_class==covid_class) & (population_file.variable==Ages)].set_index('HDNG_NM')['value'])
    
    conv_dict = dict(convenience_file.set_index(['HDONG_NM'])['conv'])
    
    """
    # case안의 행정동끼리는 인접하지 않도록 filtering
    filtered_case = []
    
    for case in tqdm(all_cases, desc='Filtering Cases'):
        if sum(sum((dist_file.loc[case,case]==0).values)) == N:
            filtered_case.append(case)
    """
    
    #각 행정동별 인접 행정동
    adjacent_dong_dict = {}
    
    for dong in tqdm(HDONGS, desc='Finding Adjacent Dongs'):
        adjacent_dong_dict[dong] = list(dist_file.loc[dist_file[dong]==0, dong].drop(dong).index)
    
    weekday_HDONG = []
    weekend_HDONG = []
    
    adjacent_dong_dict_copy = copy.deepcopy(adjacent_dong_dict)
    HDONGS_copy = copy.deepcopy(HDONGS)
    
    for i in trange(N, desc='Getting HDONG [Weekday]'):
        if i != 0:
            adjacent_dong_dict_copy = {}
            for dong in HDONGS_copy:
                tmp_df = dist_file.loc[HDONGS_copy,HDONGS_copy]
                adjacent_dong_dict_copy[dong] = list(tmp_df.loc[tmp_df[dong]==0, dong].drop(dong).index)
                
                
        hdong = HDONGS_copy[np.argmax([weekday_living_pop_dict.get(dong) + (conv_dict.get(dong)*(sum([weekday_living_pop_dict.get(adjacent_dong) for adjacent_dong in adjacent_dongs])))
                           for dong, adjacent_dongs in adjacent_dong_dict_copy.items()])]
        
        weekday_HDONG.append(hdong)

        drop_dongs = adjacent_dong_dict_copy.get(hdong)
        drop_dongs.append(hdong)
        #print(list(adjacent_dong_dict_copy.keys()))
        
        for dong in drop_dongs:
            adjacent_dong_dict_copy.pop(dong)
            HDONGS_copy.remove(dong)
        
    adjacent_dong_dict_copy = copy.deepcopy(adjacent_dong_dict)
    HDONGS_copy = copy.deepcopy(HDONGS)
    
    for i in trange(N, desc='Getting HDONG [Weekend]'):
        if i != 0:
            adjacent_dong_dict_copy = {}
            for dong in HDONGS_copy:
                tmp_df = dist_file.loc[HDONGS_copy,HDONGS_copy]
                adjacent_dong_dict_copy[dong] = list(tmp_df.loc[tmp_df[dong]==0, dong].drop(dong).index)
                
                
        hdong = HDONGS_copy[np.argmax([weekend_living_pop_dict.get(dong) + (conv_dict.get(dong)*(sum([weekend_living_pop_dict.get(adjacent_dong) for adjacent_dong in adjacent_dongs])))
                           for dong, adjacent_dongs in adjacent_dong_dict_copy.items()])]
        
        weekend_HDONG.append(hdong)

        drop_dongs = adjacent_dong_dict_copy.get(hdong)
        drop_dongs.append(hdong)
        #print(list(adjacent_dong_dict_copy.keys()))
        
        for dong in drop_dongs:
            adjacent_dong_dict_copy.pop(dong)
            HDONGS_copy.remove(dong)
            
    return weekday_HDONG, weekend_HDONG

In [39]:
MCLP(adjacent_dist, living_population, convenience_index,Ages='60U', N=4)

Getting HDONG [Weekend]: 100%|███████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 21.59it/s]


(['신당동', '명동', '월계3동', '상계2동'], ['신당동', '월계3동', '명동', '상계2동'])

In [40]:
MCLP(adjacent_dist, living_population, convenience_index, N=4)

Getting HDONG [Weekend]: 100%|███████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 36.59it/s]


(['신당동', '명동', '월계3동', '상계2동'], ['신당동', '월계3동', '명동', '상계2동'])

In [41]:
MCLP(adjacent_dist, living_population, convenience_index,Ages='30L', N=4)

Getting HDONG [Weekend]: 100%|███████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 29.41it/s]


(['신당동', '명동', '월계3동', '하계1동'], ['신당동', '월계3동', '명동', '하계1동'])

In [49]:
result = {}
for class_ in living_population.Covid_class.unique():
    for var in living_population.variable.unique():
        result[(class_,var)] = MCLP(adjacent_dist, living_population, convenience_index, Ages=var, covid_class=class_, N=4)

Getting HDONG [Weekend]: 100%|███████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 50.36it/s]


In [50]:
result

{('A', '3059'): (['신당동', '명동', '월계3동', '상계2동'], ['신당동', '월계3동', '명동', '상계2동']),
 ('A', '30L'): (['신당동', '명동', '월계3동', '하계1동'], ['신당동', '월계3동', '명동', '하계1동']),
 ('A', '60U'): (['신당동', '명동', '월계3동', '상계2동'], ['신당동', '월계3동', '명동', '상계2동']),
 ('B', '3059'): (['신당동', '명동', '월계3동', '상계2동'], ['신당동', '월계3동', '명동', '상계2동']),
 ('B', '30L'): (['신당동', '월계3동', '명동', '하계1동'], ['신당동', '월계3동', '명동', '하계1동']),
 ('B', '60U'): (['신당동', '명동', '월계3동', '상계2동'], ['신당동', '월계3동', '명동', '상계2동']),
 ('C', '3059'): (['신당동', '명동', '월계3동', '상계2동'], ['신당동', '월계3동', '명동', '상계2동']),
 ('C', '30L'): (['신당동', '월계3동', '명동', '하계1동'], ['신당동', '월계3동', '명동', '하계1동']),
 ('C', '60U'): (['신당동', '명동', '월계3동', '상계2동'], ['신당동', '월계3동', '명동', '상계2동']),
 ('D', '3059'): (['신당동', '명동', '월계3동', '상계2동'], ['신당동', '월계3동', '명동', '상계2동']),
 ('D', '30L'): (['신당동', '월계3동', '명동', '하계1동'], ['신당동', '월계3동', '명동', '하계1동']),
 ('D', '60U'): (['신당동', '명동', '월계3동', '상계2동'], ['신당동', '월계3동', '명동', '상계2동']),
 ('E', '3059'): (['신당동', '명동', '월계3동', '상계2동'], 

In [21]:
tmp_dict = {dong:0 for dong in adjacent_dist.columns}

In [23]:
[key for key,val in tmp_dict.items() if val==0]

['공릉1동',
 '공릉2동',
 '상계10동',
 '상계1동',
 '상계2동',
 '상계3.4동',
 '상계5동',
 '상계6.7동',
 '상계8동',
 '상계9동',
 '월계1동',
 '월계2동',
 '월계3동',
 '중계1동',
 '중계2.3동',
 '중계4동',
 '중계본동',
 '하계1동',
 '하계2동',
 '광희동',
 '다산동',
 '동화동',
 '명동',
 '소공동',
 '신당5동',
 '신당동',
 '약수동',
 '을지로동',
 '장충동',
 '중림동',
 '청구동',
 '필동',
 '황학동',
 '회현동']

In [20]:
list(adjacent_dist.loc[adjacent_dist['명동']==0, '명동'].drop('명동').index)

['소공동', '을지로동', '필동', '회현동']

In [ ]:
def MCLP(dist_file, population_file, convenience_file, distance='adjacent', N=5, covid_class='A', Ages='3059'):
    HDONGS=dist_file.columns.tolist()
    
    # 평일 생활인구, 주말 생활인구
    weekday_living_pop_dict = dict(population_file.loc[(population_file.dayofweek==0) & (population_file.Covid_class==covid_class) & (population_file.variable==Ages)].set_index('HDNG_NM')['value'])
    weekend_living_pop_dict = dict(population_file.loc[(population_file.dayofweek==1) & (population_file.Covid_class==covid_class) & (population_file.variable==Ages)].set_index('HDNG_NM')['value'])
    
    conv_dict = dict(convenience_file.set_index(['HDONG_NM'])['conv'])
    
    considered_dong = {dong:0 for dong in HDONGS}
    
    """
    # case안의 행정동끼리는 인접하지 않도록 filtering
    filtered_case = []
    
    for case in tqdm(all_cases, desc='Filtering Cases'):
        if sum(sum((dist_file.loc[case,case]==0).values)) == N:
            filtered_case.append(case)
    """
    
    #각 행정동별 인접 행정동
    adjacent_dong_dict = {}
    
    for dong in tqdm(HDONGS, desc='Finding Adjacent Dongs'):
        adjacent_dong_dict[dong] = list(dist_file.loc[dist_file[dong]==0, dong].drop(dong).index)
    
    weekday_HDONG = []
    weekend_HDONG = []
    
    adjacent_dong_dict_copy = copy.deepcopy(adjacent_dong_dict)
    HDONGS_copy = copy.deepcopy(HDONGS)
    
    for i in trange(N, desc='Getting HDONG [Weekday]'):
        not_considered = [dong for dong,considered in considered_dong.items() if considered==0]
        hdong = HDONGS_copy[np.argmax()]
    
    
    for i in trange(N, desc='Getting HDONG [Weekday]'):
        if i != 0:
            adjacent_dong_dict_copy = {}
            for dong in HDONGS_copy:
                tmp_df = dist_file.loc[HDONGS_copy,HDONGS_copy]
                adjacent_dong_dict_copy[dong] = list(tmp_df.loc[tmp_df[dong]==0, dong].drop(dong).index)
                
                
        hdong = HDONGS_copy[np.argmax([weekday_living_pop_dict.get(dong) + (conv_dict.get(dong)*(sum([weekday_living_pop_dict.get(adjacent_dong) for adjacent_dong in adjacent_dongs])))
                           for dong, adjacent_dongs in adjacent_dong_dict_copy.items()])]
        
        weekday_HDONG.append(hdong)

        drop_dongs = adjacent_dong_dict_copy.get(hdong)
        drop_dongs.append(hdong)
        #print(list(adjacent_dong_dict_copy.keys()))
        
        for dong in drop_dongs:
            adjacent_dong_dict_copy.pop(dong)
            HDONGS_copy.remove(dong)
        
    adjacent_dong_dict_copy = copy.deepcopy(adjacent_dong_dict)
    HDONGS_copy = copy.deepcopy(HDONGS)
    
    for i in trange(N, desc='Getting HDONG [Weekend]'):
        if i != 0:
            adjacent_dong_dict_copy = {}
            for dong in HDONGS_copy:
                tmp_df = dist_file.loc[HDONGS_copy,HDONGS_copy]
                adjacent_dong_dict_copy[dong] = list(tmp_df.loc[tmp_df[dong]==0, dong].drop(dong).index)
                
                
        hdong = HDONGS_copy[np.argmax([weekend_living_pop_dict.get(dong) + (conv_dict.get(dong)*(sum([weekend_living_pop_dict.get(adjacent_dong) for adjacent_dong in adjacent_dongs])))
                           for dong, adjacent_dongs in adjacent_dong_dict_copy.items()])]
        
        weekend_HDONG.append(hdong)

        drop_dongs = adjacent_dong_dict_copy.get(hdong)
        drop_dongs.append(hdong)
        #print(list(adjacent_dong_dict_copy.keys()))
        
        for dong in drop_dongs:
            adjacent_dong_dict_copy.pop(dong)
            HDONGS_copy.remove(dong)
            
    return weekday_HDONG, weekend_HDONG